# Generate a nested cell container structure for Alexa APL

Create a raw APL structure

In [1]:
# raw APL with background, cell containers and image scaling to be inserted
apl = """
{
  "type": "APL",
  "version": "1.3",
  "settings": {},
  "theme": "dark",
  "import": [
    {
      "name": "alexa-layouts",
      "version": "1.5.0"
    }
  ],
  "resources": [],
  "styles": {
    "cellContainerStyle": {
      "values": [
        {
          "opacity": "0.8"
        }
      ]
    },
    "cellStyle": {
      "values": [
        {
          "fontSize": "25dp",
          "textAlign": "center",
          "textAlignVertical": "center"
        }
      ]
    }
  },
  "onMount": [],
  "graphics": {},
  "commands": {},
  "layouts": {},
  "mainTemplate": {
    "parameters": [
      "payload"
    ],
    "items": [
      {
        "type": "Container",
        "width": "100%",
        "height": "100%",
        "direction": "column",
        "alignItems": "center",
        "items": [
          {
            "type": "AlexaBackground",
            "backgroundImageSource": "https://www.bildung-forschung.digital/digitalezukunft/de/bildung/mint-allianz/was-wimmelt-in-mint/_functions/wimmelbild_png.png?__blob=normal&amp;v=2&amp;.png",
            "colorOverlay": true,
            "width": WIDTH,
            "height": HEIGHT
          },
          {
                "type": "Container",
                "width": WIDTH,
                "height": HEIGHT,
                "items": [
                  CELLCONTAINERS
                  ]
                  }
            ]
          }
        ]
      }
}
"""

Calculate optimal background image scaling (like `best-fill`) for given device measures

In [2]:
def height(width):
  """compute background height given device width"""
  return width * 0.75

def width(height):
  """compute background width given device height"""
  return height * 1.33333333333

def optimum(w, h):
  """given device width and height, compute optimal background image scaling (like bestfill)"""
  wid = width(h)
  hei = height(w)
  if hei < h:
    return w, hei
  elif wid < w:
    return wid, h

In [3]:
optimum(960, 600)

(799.9999999979999, 600)

Generate the cell structure, given numbers of columns and rows, an APL draft and device width and height

In [4]:
def generate(n_rows, n_cols, apl=apl, width=1280, height=800):
    # row container draft
    s_row = """
            {
            "type": "Container",
            "id": "row-RRR",
            "width": "100%",
            "height": "RR%",
            "direction": "row",
            "items": [
            """
    # cell container draft
    s_cell = """
                {
                "type": "Container",
                "id": "row-RRR-column-CCC",
                "width": "CC%",
                "height": "100%",
                "style": "cellContainerStyle",
                "items": [
                  {
                    "type": "TouchWrapper",
                    "item": {
                      "type": "Text",
                      "id": "cell-RRR-CCC",
                      "width": "100%",
                      "height": "100%",
                      "style": "cellStyle",
                      "text": "cell-RRR-CCC",
                      "color": "#66DFFF",
                      "fontSize": 25
                    },
                    "onPress": [
                      {
                        "type": "SendEvent",
                        "arguments": [
                          "items_dict[]"
                        ]
                      }
                    ]
                  }
                  ]
                  }
                  """
    cellcontainers = """"""
    for r in range(n_rows):
        # initialize row container
        sr = s_row.replace('RRR', str(r))
        sr = sr.replace('RR%', f'{100/n_rows}%')
        temp = sr
        for c in range(n_cols):
            # add cell containers
            sc = s_cell.replace('RRR', str(r))
            sc = sc.replace('CCC', str(c))
            sc = sc.replace('CC%', f'{100/n_cols}%')
            if c == (n_cols-1):  # no comma after last cell in a row, close items list
                if r == (n_rows-1):  # no comma after last row, close items list
                    temp = temp + sc + ']}'
                else:
                    temp = temp + sc + ']},'
            else:  # comma and next cell
                temp = temp + sc + ',\n'
        cellcontainers += temp
    # insert into APL file
    output = apl.replace('CELLCONTAINERS', cellcontainers)
    # adjust background image scaling
    w, h = optimum(width, height)
    output = output.replace('WIDTH', str(w))
    output = output.replace('HEIGHT', str(h))
    with open('cellcontainer.txt', 'w', encoding='utf-8') as f:
        f.write(output)

Generate a 7x8 cell structure for a 1280x800 DP device

In [5]:
generate(7,8)